In [2]:
import pandas as pd
from api import get_api
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
from tqdm import tqdm

In [3]:
# Cluster vrp 정보 가져오기
solution = pd.read_csv('solution_cluster_info.csv')
delete = []
for i in range(len(solution)):
    if(solution.iloc[i,2] == 50.0):
        delete.append(i)
print(solution)
delete.sort(reverse=True)
print(delete)
for i in delete:
    solution = solution.drop([solution.index[i]])
    
#     
solution = pd.DataFrame(solution)
solution = solution.reset_index(drop=True)
solution

    Unnamed: 0   0   1     2     3     4     5     6     7     8     9    10
0            0  50  50   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
1            1  50   1  41.0  23.0  39.0  50.0   NaN   NaN   NaN   NaN   NaN
2            2  50  15  20.0  22.0   6.0  43.0  35.0  50.0   NaN   NaN   NaN
3            3  50  18  40.0   4.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN
4            4  50  19  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
5            5  50  45  49.0  26.0  38.0   0.0  27.0  37.0   8.0  50.0   NaN
6            6  50  31  14.0   7.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN
7            7  50  46  17.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN
8            8  50  32  13.0   9.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN
9            9  50   5  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
10          10  50  50   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN
11          11  50  16  25.0  50.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,1,50,1,41.0,23.0,39.0,50.0,NaN,NaN,NaN,NaN,NaN
1,2,50,15,20.0,22.0,6.0,43.0,35.0,50.0,NaN,NaN,NaN
2,3,50,18,40.0,4.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,50,19,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,50,45,49.0,26.0,38.0,0.0,27.0,37.0,8.0,50.0,NaN
5,6,50,31,14.0,7.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
6,7,50,46,17.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,50,32,13.0,9.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN
8,9,50,5,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,50,16,25.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 지점 정보 가져오기
spot = pd.read_csv('clusterinfo.csv')

In [5]:
def find(cNum):
    cNum = cNum[2:]
    result = pd.DataFrame()
    for i in cNum:
        if(i==50):
            break
        temp = spot[spot['cluster']==i]
        result = pd.concat([result, temp],ignore_index=True,axis=0)
        result = result[['lat','lng']]
    return result

In [6]:
solutionList = solution.values.tolist()
arrive = {'lat': 35.229546, 'lng' : 126.847516}

In [7]:
def calcdistance(A,B):
    return abs(A['lat']-B['lat']) + abs(A['lng']-B['lng'])

In [8]:
def calc_distance(originX,originY,destinationX, destinationY):
    # origin and destination should be coordinate
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {'origin': str(originX) +','+str(originY),
              'destination': str(destinationX) + ','+str(destinationY)}
    header = {'Authorization': 'KakaoAK ' + get_api()}

    route = requests.get(url, params=params, headers=header).json()
#     print(route)

    if route['routes'][0]['result_code'] == 104:
        return 0
    else:
        return(route['routes'][0]['summary']['duration'])

In [9]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [10]:
print('start')

start


In [ ]:
answer = []

sL = tqdm(solutionList)
for i in sL:
    newDF = pd.DataFrame(find(i))
    if(len(newDF) <= 0):
        continue
    newDF.append(arrive,ignore_index=True)
    result = []
    for j in tqdm(range(len(newDF)-1)):
        temp = []
        for k in range(len(newDF)):
            distance = calcdistance(newDF.iloc[j],newDF.iloc[k]) * 1000000
            temp.append(distance)
        result.append(temp)
    result.append([0]*len(newDF))
    
#     print(len(result),len(result[0]))
    def create_data_model():
        data = {}
        data['distance_matrix'] = result
        data['num_vehicles'] = 3
        data['depot'] = len(result)
        return data
    
    # Instantiate the data problem.

    data = create_data_model()
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        100000,  # no slack
        30000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    def print_solution_only_num():
        output = []
        for vehicle_id in range(data['num_vehicles']):
            Output = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                Output.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            Output.append(manager.IndexToNode(index))  
            output.append(Output)
        return output
    
    if solution:
        sol = print_solution_only_num()
        answer.append(sol)
        print(sol)
    else:
        print('No solution found !')

  0%|                                                                                                                                                                                                                  | 0/13 [00:00<?, ?it/s]/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1708/2547019778.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)

  8%|███████████████▌                                                                                                                                                                                          | 1/13 [00:02<00:26,  2.23s/it]

218 218
[[0, 218], [218, 218], [218, 217, 216, 215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1708/2547019778.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉     | 153/157 [00:00<00:00, 160.84it/s]

In [16]:
answerDF = pd.read_csv('allDF')

In [57]:
answerDF

,0,1,2
0,"[217, 112, 169, 192, 168, 217]","[217, 115, 116, 113, 114, 186, 208, 178, 179, ...","[217, 45, 66, 109, 110, 39, 77, 65, 46, 55, 60..."
1,"[157, 156, 155, 154, 153, 39, 43, 41, 42, 40, ...","[157, 5, 31, 12, 23, 25, 17, 16, 29, 10, 4, 6,...","[157, 2, 1, 0, 157]"
2,"[194, 27, 55, 16, 41, 44, 57, 21, 71, 4, 68, 7...","[194, 83, 84, 85, 88, 96, 99, 100, 102, 108, 1...","[194, 12, 52, 22, 66, 23, 29, 50, 53, 40, 26, ..."
3,"[39, 1, 13, 4, 3, 7, 2, 11, 29, 23, 0, 33, 8, ...","[39, 26, 12, 14, 16, 22, 39]","[39, 28, 27, 15, 21, 36, 38, 35, 32, 31, 24, 5..."
4,"[218, 13, 14, 159, 163, 148, 153, 152, 147, 14...","[218, 0, 1, 2, 9, 11, 10, 8, 6, 5, 7, 12, 202,...","[218, 4, 3, 60, 27, 32, 40, 65, 66, 62, 20, 78..."
5,"[127, 2, 8, 12, 1, 4, 5, 10, 3, 9, 11, 0, 13, ...","[127, 23, 39, 40, 59, 74, 51, 37, 69, 70, 55, ...","[127, 28, 31, 33, 41, 46, 49, 50, 53, 63, 60, ..."
6,"[88, 14, 7, 1, 5, 4, 2, 9, 11, 0, 12, 8, 6, 3,...","[88, 39, 76, 58, 17, 41, 44, 48, 59, 67, 24, 6...","[88, 34, 38, 52, 66, 73, 72, 62, 86, 88]"
7,"[158, 2, 5, 12, 46, 47, 53, 62, 64, 75, 79, 81...","[158, 101, 99, 98, 95, 84, 83, 85, 89, 93, 96,...","[158, 86, 87, 92, 91, 102, 103, 90, 82, 97, 48..."
8,"[90, 66, 31, 57, 69, 44, 40, 7, 3, 86, 65, 54,...","[90, 34, 36, 37, 71, 63, 58, 48, 45, 80, 8, 15...","[90, 1, 2, 5, 6, 9, 10, 11, 13, 21, 26, 29, 33..."
9,"[141, 98, 70, 113, 110, 101, 73, 139, 133, 126...","[141, 3, 4, 5, 8, 10, 14, 15, 18, 21, 22, 31, ...","[141, 57, 67, 11, 19, 37, 56, 50, 40, 35, 28, ..."


In [81]:
m = folium.Map( location = [35.212616,126.842246],zoom_start=13)

solList = []
for i in solutionList:
    if(i[2] == 50.0):
        continue
    solList.append(i)
    
solutionList = solList

In [89]:
# sL = tqdm(solutionList)

for i in range(len(answerDF)):

    for j in range(len(answerDF.iloc[i])):
        print(i,j)
        tempList = answerDF.iloc[i,j]
        tempList = tempList[1:]
        tempList = tempList[:-1]
        tempList = tempList.replace(" ","")
        tempList = tempList.split(',')

        if(len(tempList) <= 2):
            continue        
            
        newDF = pd.DataFrame(find(solutionList[i]))
        if(int(len(newDF)) <= 0):
            continue
        newDF.append(arrive,ignore_index=True)
        print(tempList[-1])
        print(newDF.iloc[int(tempList[-1])])
        for k in range(1,len(tempList)-1):
            folium.PolyLine(locations=[[newDF.iloc[int(tempList[k])]['lat'],newDF.iloc[int(tempList[k])]['lng']], [newDF.iloc[int(tempList[k])+1]['lat'],newDF.iloc[int(tempList[k])+1]['lng']]]
                            ,tooltip='Polyline'
                           ).add_to(m)


0 0
217
lat     35.210275
lng    126.841962
Name: 217, dtype: float64
0 1
217
lat     35.210275
lng    126.841962
Name: 217, dtype: float64
0 2
217
lat     35.210275
lng    126.841962
Name: 217, dtype: float64
1 0
157
lat     35.160006
lng    126.864456
Name: 157, dtype: float64
1 1
157
lat     35.160006
lng    126.864456
Name: 157, dtype: float64
1 2
157
lat     35.160006
lng    126.864456
Name: 157, dtype: float64
2 0
194
lat     35.210000
lng    126.881398
Name: 194, dtype: float64
2 1
194
lat     35.210000
lng    126.881398
Name: 194, dtype: float64
2 2
194
lat     35.210000
lng    126.881398
Name: 194, dtype: float64
3 0
39
lat     35.207917
lng    126.865400
Name: 39, dtype: float64
3 1
39
lat     35.207917
lng    126.865400
Name: 39, dtype: float64
3 2
39
lat     35.207917
lng    126.865400
Name: 39, dtype: float64
4 0
218
lat     35.207306
lng    126.862428
Name: 218, dtype: float64
4 1
218
lat     35.207306
lng    126.862428
Name: 218, dtype: float64
4 2
218
lat     35.207306


/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(ar

88
lat     35.219443
lng    126.842818
Name: 88, dtype: float64
7 0
158
lat     35.211333
lng    126.833902
Name: 158, dtype: float64
7 1
158
lat     35.211333
lng    126.833902
Name: 158, dtype: float64
7 2
158
lat     35.211333
lng    126.833902
Name: 158, dtype: float64
8 0
90
lat     35.211722
lng    126.837374
Name: 90, dtype: float64
8 1
90
lat     35.211722
lng    126.837374
Name: 90, dtype: float64
8 2
90
lat     35.211722
lng    126.837374
Name: 90, dtype: float64
9 0
141
lat     35.213194
lng    126.836902
Name: 141, dtype: float64
9 1
141
lat     35.213194
lng    126.836902
Name: 141, dtype: float64
9 2
141
lat     35.213194
lng    126.836902
Name: 141, dtype: float64
10 0
164
lat     35.208972
lng    126.842235
Name: 164, dtype: float64
10 1
164
lat     35.208972
lng    126.842235
Name: 164, dtype: float64
10 2
164
lat     35.208972
lng    126.842235
Name: 164, dtype: float64
11 0
341
lat     35.219943
lng    126.849067
Name: 341, dtype: float64
11 1
341
lat     35.219943
l

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)
/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_1255/2053527509.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(ar

In [84]:
m